# Matrix Encryption
Makenna Worley and Nic Van Der Werf

An exploration into encrypting data through matrices transformations and modular arithmetic

In [46]:
# Importing and declaring global variables
import numpy as np
import math
import random

from ipywidgets import widgets
from IPython.display import display

MODULO = 59
ALPHABET = "ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz .,!?&\n"
# print(len(ALPHABET))

KEY_MATRIX_2x2 = np.array([
    [3, 2],
    [5, 7]
])

text1 = open('text/one.txt').read()
text2 = open('text/two.txt').read()
text3 = open('text/our_super_secret_message.txt').read()

Functions that will help us with encrypting and decrypting our secret messages!

In [47]:
def generate_key_matrix(n, modulo):
    while True:
        key_matrix = np.random.randint(0, modulo, size=(n, n))

        # Calculating the determinant with Mod
        a, b = key_matrix[0]
        c, d = key_matrix[1]
        det = (a*d - b*c) % modulo
        if math.gcd(det, modulo) == 1:
            try:
                inverse_key_matrix = inverse_2x2_mod(key_matrix, modulo)
                return key_matrix, inverse_key_matrix
            except ValueError:
                continue
        else:
            print("Failed Iteration")


def generate_key_matrix_nxn(n, modulo):
    while True:
        key_matrix = np.random.randint(0, modulo, size=(n, n))
        det = int(round(np.linalg.det(key_matrix))) % modulo
        if math.gcd(det, modulo) == 1:
            try:
                inverse_key_matrix = inverse_nxn_mod(key_matrix, modulo)
                return key_matrix, inverse_key_matrix
            except ValueError:
                continue
        else:
            # All matrices must be invertible
            pass


def inverse_2x2_mod(matrix, modulo):
    a, b = matrix[0]
    c, d = matrix[1]
    det = int((a*d - b*c) % modulo)
    if math.gcd(det, modulo) != 1:
        raise ValueError("Matrix not invertible modulo {}".format(modulo))
    det_inv = pow(det, -1, modulo)
    return (det_inv * np.array([[d, -b],
                                [-c, a]])) % modulo


# Inverting matrices with >2 rows and columns requires GCD since it is not guaranteed that rows and columns will be the same number, so we need to look at the coprime of them
def inverse_nxn_mod(matrix, modulo):
    A = np.array(matrix, dtype=int) % modulo
    n = A.shape[0]

    I = np.eye(n, dtype=int)

    # Perform Gaussian elimination
    for i in range(n):
        # Find pivot (row with non-zero element in column i)
        for r in range(i, n):
            if A[r, i] % modulo != 0:
                # Swap pivot row if necessary
                if r != i:
                    A[[i, r]] = A[[r, i]]
                    I[[i, r]] = I[[r, i]]
                break
        else:
            # No pivot found, matrix not invertible
            raise ValueError("Matrix not invertible modulo {}".format(modulo))

        # Normalize pivot row
        inv_pivot = pow(int(A[i, i]), -1, modulo)
        A[i] = (A[i] * inv_pivot) % modulo
        I[i] = (I[i] * inv_pivot) % modulo

        # Eliminate other rows
        for r in range(n):
            if r != i:
                factor = A[r, i] % modulo
                if factor != 0:
                    A[r] = (A[r] - factor * A[i]) % modulo
                    I[r] = (I[r] - factor * I[i]) % modulo

    return I % modulo


def encrypt(plaintext, key_matrix, modulo, alphabet):
    # Map chars to numbers from 0 to len(alphabet)-1
    char_to_num = {char: idx for idx, char in enumerate(alphabet)}
    num_to_char = {v: k for k, v in char_to_num.items()}
    padding_val = len(alphabet)  # Use a padding value outside the char range

    # Convert plaintext to numbers
    numbers = [char_to_num[c] for c in plaintext if c in char_to_num]

    # Pad if necessary
    if len(numbers) % 2 != 0:
        numbers.append(padding_val)

    plaintext_matrix = np.array(numbers).reshape(-1, 2).T
    ciphertext_matrix = (key_matrix.dot(plaintext_matrix) % modulo).T

    # Convert back to chars (skip padding_val when converting back)
    ciphertext = []
    for pair in ciphertext_matrix:
        for num in pair:
            if num != padding_val:
                if num in num_to_char:
                    ciphertext.append(num_to_char[num])
                else:
                    # If something unexpected, place a placeholder
                    ciphertext.append('?')
    return ''.join(ciphertext), ciphertext_matrix


def encrypt_nxn(plaintext, key_matrix, modulo, alphabet):
    # Map chars to numbers from 0 to len(alphabet)-1
    n = key_matrix.shape[0]
    char_to_num = {char: idx for idx, char in enumerate(alphabet)}
    num_to_char = {v: k for k, v in char_to_num.items()}
    padding_val = len(alphabet)  # Use a padding value outside the char range

    # Convert plaintext to numbers
    numbers = [char_to_num[c] for c in plaintext if c in char_to_num]

    # Pad if necessary
    remainder = len(numbers) % n
    if remainder != 0:
        numbers.extend([padding_val] * (n - remainder))

    plaintext_matrix = np.array(numbers).reshape(-1, n).T
    ciphertext_matrix = (key_matrix.dot(plaintext_matrix) % modulo).T

    # Convert back to chars (skip padding_val when converting back)
    ciphertext = []
    for pair in ciphertext_matrix:
        for num in pair:
            if num != padding_val:
                if num in num_to_char:
                    ciphertext.append(num_to_char[num])
                else:
                    # If something unexpected, place a placeholder
                    ciphertext.append('?')
    return ''.join(ciphertext), ciphertext_matrix


def decrypt(ciphertext, inverse_key_matrix, modulo, alphabet):
    char_to_num = {char: idx for idx, char in enumerate(alphabet)}
    num_to_char = {v: k for k, v in char_to_num.items()}
    padding_val = len(alphabet)

    numbers = [char_to_num[c] for c in ciphertext if c in char_to_num]
    if len(numbers) % 2 != 0:
        numbers.append(padding_val)

    cipher_matrix = np.array(numbers).reshape(-1, 2)
    decrypted_matrix = (inverse_key_matrix.dot(cipher_matrix.T) % modulo).T

    # Convert back to text and remove padding
    plaintext_chars = []
    for pair in decrypted_matrix:
        for num in pair:
            if num == padding_val:
                # Padding encountered, skip it
                continue
            if num in num_to_char:
                plaintext_chars.append(num_to_char[num])
            else:
                plaintext_chars.append('?')
    return ''.join(plaintext_chars)


def decrypt_nxn(ciphertext, inverse_key_matrix, modulo, alphabet):
    n = inverse_key_matrix.shape[0]
    char_to_num = {char: idx for idx, char in enumerate(alphabet)}
    num_to_char = {v: k for k, v in char_to_num.items()}
    padding_val = len(alphabet)

    numbers = [char_to_num[c] for c in ciphertext if c in char_to_num]
    remainder = len(numbers) % n
    if remainder != 0:
        numbers.extend([padding_val] * (n - remainder))

    cipher_matrix = np.array(numbers).reshape(-1, n)
    decrypted_matrix = (inverse_key_matrix.dot(cipher_matrix.T) % modulo).T

    # Convert back to text and remove padding
    plaintext_chars = []
    for pair in decrypted_matrix:
        for num in pair:
            if num == padding_val:
                # Padding encountered, skip it
                continue
            if num in num_to_char:
                plaintext_chars.append(num_to_char[num])
            else:
                plaintext_chars.append('?')
    dec = ''.join(plaintext_chars)
    return dec.rstrip('A')


def generate_random_string(alphabet, l):
    return ''.join(random.choice(alphabet) for _ in range(l))


def test_encryption_decryption(l):
    plaintext = generate_random_string(ALPHABET, l)
    ciphertext, ciphertext_matrix = encrypt(plaintext, KEY_MATRIX_2x2, MODULO, ALPHABET)
    decrypted_text = decrypt(ciphertext, inverse_key_matrix, MODULO, ALPHABET)
    return plaintext, decrypted_text, plaintext == decrypted_text

First testing our matrix to make sure it is invertible and therefore useable for our purposes.

In [48]:
inverse_key_matrix = inverse_2x2_mod(KEY_MATRIX_2x2, MODULO)

for idx in range(1, 51):
    plaintext, decrypted_text, result = test_encryption_decryption(200)
    if result:
        print(f"Test {idx}: True")
    else:
        print(f"Test {idx}: False")
        print(f"  Original: {plaintext}")
        print(f"  Decrypted: {decrypted_text}")

Test 1: True
Test 2: True
Test 3: True
Test 4: True
Test 5: True
Test 6: True
Test 7: True
Test 8: True
Test 9: True
Test 10: True
Test 11: True
Test 12: True
Test 13: True
Test 14: True
Test 15: True
Test 16: True
Test 17: True
Test 18: True
Test 19: True
Test 20: True
Test 21: True
Test 22: True
Test 23: True
Test 24: True
Test 25: True
Test 26: True
Test 27: True
Test 28: True
Test 29: True
Test 30: True
Test 31: True
Test 32: True
Test 33: True
Test 34: True
Test 35: True
Test 36: True
Test 37: True
Test 38: True
Test 39: True
Test 40: True
Test 41: True
Test 42: True
Test 43: True
Test 44: True
Test 45: True
Test 46: True
Test 47: True
Test 48: True
Test 49: True
Test 50: True


Now let's look at our encrypting a message using our predefined KEY_MATRIX_2x2

In [55]:
MESSAGE = text1

inverse_key_matrix = inverse_2x2_mod(KEY_MATRIX_2x2, MODULO)
print(f"Key Matrix:\n{KEY_MATRIX_2x2}")
print(f"Inverse Key Matrix:\n{inverse_key_matrix}")

ciphertext, ciphertext_matrix = encrypt(MESSAGE, KEY_MATRIX_2x2, MODULO, ALPHABET)
print(f"Ciphertext Matrix:\n{ciphertext_matrix}")

decrypted_text = decrypt(ciphertext, inverse_key_matrix, MODULO, ALPHABET)
print(f"Ciphertext: {ciphertext}")
print(f"Condition: {np.linalg.cond(KEY_MATRIX_2x2)}")

original_text = MESSAGE

original_button = widgets.Button(description="Show Original Text")
decrypted_button = widgets.Button(description="Show Decrypted Text")
original_output = widgets.Output()
decrypted_output = widgets.Output()

def show_original_text(button):
    with original_output:
        original_output.clear_output()
        print(original_text)

def show_decrypted_text(button):
    with decrypted_output:
        decrypted_output.clear_output()
        print(decrypted_text)

original_button.on_click(show_original_text)
decrypted_button.on_click(show_decrypted_text)

display(original_button, original_output)
display(decrypted_button, decrypted_output)


Key Matrix:
[[3 2]
 [5 7]]
Inverse Key Matrix:
[[ 6 32]
 [21 11]]
Ciphertext Matrix:
[[22  9]
 [ 8 31]
 [47 33]
 [18 56]
 [57  2]
 [36 42]
 [17 42]
 [13  6]
 [ 5 56]
 [40 12]]
Ciphertext: WJIfvhS?&CkqRqNGF?oM
Condition: 7.780565547548909


Button(description='Show Original Text', style=ButtonStyle())

Output()

Button(description='Show Decrypted Text', style=ButtonStyle())

Output()

In [59]:
MESSAGE = text1

key_matrix, inverse_key_matrix = generate_key_matrix(2, MODULO)

print(f"Key Matrix:\n{key_matrix}")
print(f"Inverse Key Matrix:\n{inverse_key_matrix}")

ciphertext, ciphertext_matrix = encrypt(MESSAGE, key_matrix, MODULO, ALPHABET)
print(f"Ciphertext Matrix:\n{ciphertext_matrix}")

decrypted_text = decrypt(ciphertext, inverse_key_matrix, MODULO, ALPHABET)
print(f"Ciphertext: {ciphertext}")
print(f"Condition: {np.linalg.cond(key_matrix)}")

original_text = MESSAGE

original_button = widgets.Button(description="Show Original Text")
decrypted_button = widgets.Button(description="Show Decrypted Text")
original_output = widgets.Output()
decrypted_output = widgets.Output()

def show_original_text(button):
    with original_output:
        original_output.clear_output()
        print(original_text)

def show_decrypted_text(button):
    with decrypted_output:
        decrypted_output.clear_output()
        print(decrypted_text)

original_button.on_click(show_original_text)
decrypted_button.on_click(show_decrypted_text)

display(original_button, original_output)
display(decrypted_button, decrypted_output)

Key Matrix:
[[48 55]
 [29 20]]
Inverse Key Matrix:
[[52 34]
 [19 54]]
Ciphertext Matrix:
[[39 36]
 [35 43]
 [ 1 17]
 [22 50]
 [39 46]
 [34 39]
 [52 22]
 [40 37]
 [25 57]
 [45 12]]
Ciphertext: nkjrBRWynuin WolZ&tM
Condition: 10.248885104412558


Button(description='Show Original Text', style=ButtonStyle())

Output()

Button(description='Show Decrypted Text', style=ButtonStyle())

Output()

In [64]:
MESSAGE = text3
KEY_MATRIX_SIZE = 7

key_matrix, inverse_key_matrix = generate_key_matrix_nxn(KEY_MATRIX_SIZE, MODULO)

ciphertext, ciphertext_matrix = encrypt_nxn(MESSAGE, key_matrix, MODULO, ALPHABET)
print("Plain Text Length:", len(text1))
print("Ciphertext Length:", len(ciphertext))

print(f"Key Matrix:\n{key_matrix}")
print(f"Inverse Key Matrix:\n{inverse_key_matrix}")

decrypted_text = decrypt_nxn(ciphertext, inverse_key_matrix, MODULO, ALPHABET)
print(f"Ciphertext: {ciphertext}")

original_text = MESSAGE

original_button = widgets.Button(description="Show Original Text")
decrypted_button = widgets.Button(description="Show Decrypted Text")
original_output = widgets.Output()
decrypted_output = widgets.Output()

def show_original_text(button):
    with original_output:
        original_output.clear_output()
        print(original_text)

def show_decrypted_text(button):
    with decrypted_output:
        decrypted_output.clear_output()
        print(decrypted_text)

original_button.on_click(show_original_text)
decrypted_button.on_click(show_decrypted_text)

display(original_button, original_output)
display(decrypted_button, decrypted_output)

Plain Text Length: 20
Ciphertext Length: 1701
Key Matrix:
[[23 21 40 30 17 52 58]
 [38 22 40 17 21 40 51]
 [48  9 56 55  2 58 31]
 [ 6 53 32 28 10 23 27]
 [52  4  5 39 48 51 35]
 [ 4 45  1 43 13 10  5]
 [18 42 17 30 25 15 44]]
Inverse Key Matrix:
[[26 12 57 39  7  7 53]
 [54 21 29  3 13 46 49]
 [27  5 20 58 42 37 51]
 [36 16 42  5 35 22 48]
 [39 10 49 43 26 26  8]
 [24 58 51 11 14 36 39]
 [20 40 31 13 27 35 45]]
Ciphertext: UoWEmZdlJy .OwkPWOdi.&sV,v,ElXsquKLWFqTeCws&zxnUVkQ&FSwtz
w&W?!xcTbOM?ecRBvdr
A?tOUwyJYWWNxYhMQ.vYZSrVLqFC KW
nEXektkTRUVqOVTK.
jOHyM ,HMX.l
MnWS lF,u&jfg?YggcFmHeeiwxKgANiwuHFcXUBxFa,PTNZLrDiqgQrQAZoGWs OquNa,EurIiFUIYsZR&YYwAjNxU?zHeJ&IJNaHpwoKahp,Dv OquNa,rPnAs&TRixAFDJFDJ!p!rO.RVOXlYwAjNxUskerrZPeKnICqjbfewkvJ OquNa,gAH?duUromLov
YwAjNxUNuxTlK
?dFn.L
aCCRekOReZzcgIQAGxQNpqHEuHBXbMd!jJe&ss!QHVh.lfaOuG.QEEevkJysWKL, ,GIpYMNRbCQOUSVkuz.Labmmwmw,JRdu jDheatEjwEPkvDdruoMZL?JgWzzgAUHipkj
aJMCKMRqENPDaa,YnvzMwAVkc?ldqIFXHUlfPTFkchuWI!Sk
dmtDSb&oVNrxrcJc&lsmHeeiwxKus&Tw

Button(description='Show Original Text', style=ButtonStyle())

Output()

Button(description='Show Decrypted Text', style=ButtonStyle())

Output()